In [40]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from langdetect import detect
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score

## Data Processing
**Importing Datasets**

In [41]:
#importing the datasets
data1_fake = pd.read_csv('./Data_RealFake/Dataset1/Fake1.csv')
data1_true = pd.read_csv('./Data_RealFake/Dataset1/True1.csv')

dtset1_fake = pd.DataFrame(data1_fake)
dtset1_true = pd.DataFrame(data1_true)

Changing the labels: fake = 0 and real = 1

In [42]:
dtset1_fake['label']=0
dtset1_true['label']=1

pd.DataFrame.head(dtset1_fake)
pd.DataFrame.head(dtset1_true)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [43]:
print(dtset1_fake.shape)
print(dtset1_true.shape)

(23481, 5)
(21417, 5)


Concatinating Datasets

In [44]:
dataset1 = pd.concat([dtset1_fake, dtset1_true])
pd.DataFrame.head(dataset1)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [45]:
dataset1.label.value_counts()

0    23481
1    21417
Name: label, dtype: int64

Combining Title and Text

In [46]:
dataset1['content'] = dataset1['title']+" "+dataset1['text']

In [47]:
pd.DataFrame.head(dataset1)

,title,text,subject,date,label,content
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis Just Called Out Donald Trump Dur...


Dataset shuffle

In [48]:
#dataset1 = shuffle(dataset1)
#dataset1[:10]

Text Cleaning

In [49]:
content = dataset1.content
print(content)

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorit...
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: content, Length: 44898, dtype: object


In [50]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed = re.sub('[^a-zA-Z]',' ', content)
    stemmed = stemmed.lower()
    stemmed = stemmed.split()
    stemmed = [port_stem.stem(word) for word in stemmed if not word in stopwords.words('english')]
    stemmed = ' '.join(stemmed)
    return stemmed

In [51]:
dataset1['content'] = dataset1['content'].apply(stemming)
pd.DataFrame.head(dataset1)

KeyboardInterrupt: 

In [52]:
X = dataset1['content'].values

In [53]:
#TfIdf Vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

print(X)

  (0, 121245)	0.03493138105375333
  (0, 121196)	0.0925351818248464
  (0, 120806)	0.04705882505914014
  (0, 120794)	0.22397695617886632
  (0, 120735)	0.0690169893140659
  (0, 119813)	0.02553530393651721
  (0, 119630)	0.02175717888185537
  (0, 119510)	0.018464753242459764
  (0, 119490)	0.028112715715497144
  (0, 119395)	0.0317136712548268
  (0, 119394)	0.02050601156045364
  (0, 119067)	0.16374976719209164
  (0, 118719)	0.02225305337687285
  (0, 118542)	0.01892551586770723
  (0, 118487)	0.0290687943620557
  (0, 118294)	0.012099355686385777
  (0, 118245)	0.02475999299710875
  (0, 118026)	0.012020042873253367
  (0, 118012)	0.02126194805475316
  (0, 118001)	0.0716536150036487
  (0, 117994)	0.049429098531608144
  (0, 117955)	0.0716536150036487
  (0, 117154)	0.03572879442009054
  (0, 116518)	0.026315763296670753
  (0, 115624)	0.05029958970114686
  :	:
  (44897, 21939)	0.1057515015694217
  (44897, 18233)	0.08615292828675535
  (44897, 18169)	0.06374407852186688
  (44897, 17874)	0.030497551736619

**Train Test Split**

In [54]:
X = dataset1['content'].values
y = dataset1['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=80)

print(X_train.shape)
print(X_test.shape)


(35918,)
(8980,)


In [55]:
X_train.shape

(35918,)

In [56]:
X_test[:20]

X_test.shape

(8980,)

In [57]:
y_train[:20]
y_train.shape

(35918,)

In [58]:
y_test.shape

(8980,)

In [59]:
len(X_train)

35918

In [60]:
len(X_test)

8980

**CountVectorizer**

In [63]:
#covector = CountVectorizer()
#X_traincovector = covector.transform(X_train)
#X_testcovector = covector.transform(X_test)

In [64]:
print(X_traincovector)

  (0, 265)	1
  (0, 434)	1
  (0, 559)	1
  (0, 1143)	1
  (0, 2294)	1
  (0, 2874)	1
  (0, 3265)	1
  (0, 6689)	1
  (0, 6962)	3
  (0, 7856)	1
  (0, 8193)	1
  (0, 8536)	1
  (0, 8570)	1
  (0, 9561)	1
  (0, 9708)	1
  (0, 10414)	5
  (0, 10548)	1
  (0, 10553)	1
  (0, 12039)	1
  (0, 13349)	1
  (0, 15608)	1
  (0, 16165)	1
  (0, 16928)	1
  (0, 17531)	1
  (0, 18247)	1
  :	:
  (35917, 62653)	1
  (35917, 62787)	3
  (35917, 63270)	1
  (35917, 63280)	1
  (35917, 64121)	1
  (35917, 64129)	1
  (35917, 64589)	2
  (35917, 65040)	1
  (35917, 65242)	1
  (35917, 65334)	1
  (35917, 65839)	1
  (35917, 65883)	1
  (35917, 68744)	1
  (35917, 68812)	1
  (35917, 69471)	1
  (35917, 70222)	1
  (35917, 70598)	2
  (35917, 70981)	1
  (35917, 71133)	1
  (35917, 71761)	1
  (35917, 75547)	1
  (35917, 75975)	1
  (35917, 76587)	1
  (35917, 76707)	1
  (35917, 79805)	1


In [65]:
print(X_testcovector)

  (0, 308)	1
  (0, 313)	3
  (0, 607)	1
  (0, 1073)	1
  (0, 1263)	3
  (0, 1416)	1
  (0, 1604)	1
  (0, 1707)	2
  (0, 2024)	1
  (0, 3178)	1
  (0, 3816)	1
  (0, 4923)	5
  (0, 5050)	1
  (0, 5433)	1
  (0, 5590)	1
  (0, 6355)	1
  (0, 6765)	1
  (0, 6843)	1
  (0, 7038)	1
  (0, 7075)	1
  (0, 7093)	7
  (0, 7106)	2
  (0, 7701)	2
  (0, 7857)	4
  (0, 7868)	1
  :	:
  (8979, 28979)	1
  (8979, 29731)	1
  (8979, 30162)	2
  (8979, 30173)	1
  (8979, 30221)	1
  (8979, 30438)	1
  (8979, 31449)	1
  (8979, 32377)	1
  (8979, 33564)	1
  (8979, 33679)	1
  (8979, 34466)	1
  (8979, 34591)	1
  (8979, 35403)	1
  (8979, 35609)	1
  (8979, 35709)	1
  (8979, 35755)	1
  (8979, 36147)	1
  (8979, 37515)	1
  (8979, 37757)	4
  (8979, 39579)	1
  (8979, 40178)	1
  (8979, 40370)	1
  (8979, 40391)	1
  (8979, 40893)	1
  (8979, 40989)	1


**Linear Regression**

In [66]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_testcovector, y_test)
clf.predict(X_testcovector)

array([-4.32747577e-05,  9.99408755e-01,  1.00000527e+00, ...,
        1.00041864e+00,  1.39414430e-04,  3.57170349e-04])

In [67]:
clf.score(X_testcovector, y_test)

0.9999977009986402

**Logistic Regression**

In [68]:
logreg = LogisticRegression()
logreg.fit(X_testcovector, y_test)

LogisticRegression()

In [69]:
logreg.predict(X_testcovector)

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

**Accuracy**